# LINE Notify

In [171]:
import os
import requests

In [172]:
notify_url = "https://notify-api.line.me/api/notify"
access_token = open('access_token').read()
headers = {'Authorization': 'Bearer ' + access_token}

In [6]:
message = """テスト"""
payload = {'message': message}
r = requests.post(notify_url, headers=headers, params=payload,)

# スクレピング①

In [11]:
from bs4 import BeautifulSoup
from lxml import html
import urllib3

In [12]:
# Web Connection
http = urllib3.PoolManager()

In [31]:
url = 'https://reserve.tokyodisneyresort.jp/restaurant/calendar/?useDate=20220515&searchUseDate=20220515&adultNum=2&searchAdultNum=2&childNum=2&searchChildNum=2&stretcherCount=0&searchStretcherCount=0&wheelchairCount=0&searchWheelchairCount=0&nameCd=RBBY0&searchNameCd=RBBY0&contentsCd=04&childAgeInform=06U%7C04%7C&searchChildAgeInform=06U%7C04%7C&mealDivList=3&searchMealDivList=3&searchKeyword=&reservationStatus=0'
# xpath_data1 = '/html/body/div[2]/div[1]/div[3]/div[2]/div[2]/div/div/ul[1]/li[1]/p'
xpath_data1 = '/html/body/div[2]/div[1]/div[3]/div[2]/div[2]/div/div'

In [32]:
lxml_coverted_data = html.fromstring(
    str(
        BeautifulSoup(
            http.request('GET', url).data,'html.parser'
        )
    )
)

data1 = lxml_coverted_data.xpath(xpath_data1)
data1

[]

In [36]:
lxml_coverted_data.text_content()

'\nAccess Denied\n\nAccess Denied\n \nYou don\'t have permission to access "http://reserve.tokyodisneyresort.jp/restaurant/calendar/?" on this server.\nReference #18.5faac117.1650032374.11de9a1a\n\n'

In [38]:
str(lxml_coverted_data)

'<Element html at 0x7efc67502270>'

# スクレピング②

In [12]:
from lxml import html

In [9]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.select import Select

In [120]:
import time

In [157]:
url = 'https://reserve.tokyodisneyresort.jp/restaurant/calendar/?useDate=20220515&searchUseDate=20220515&adultNum=2&searchAdultNum=2&childNum=2&searchChildNum=2&stretcherCount=0&searchStretcherCount=0&wheelchairCount=0&searchWheelchairCount=0&nameCd=RBBY0&searchNameCd=RBBY0&contentsCd=04&childAgeInform=06U%7C04%7C&searchChildAgeInform=06U%7C04%7C&mealDivList=3&searchMealDivList=3&searchKeyword=&reservationStatus=0'


# url = 'https://reserve.tokyodisneyresort.jp/restaurant/calendar/?useDate=20220513&searchUseDate=20220513&adultNum=2&searchAdultNum=2&childNum=2&searchChildNum=2&stretcherCount=0&searchStretcherCount=0&wheelchairCount=0&searchWheelchairCount=0&nameCd=RBPP0&searchNameCd=RBPP0&contentsCd=04&childAgeInform=06U%7C04%7C&searchChildAgeInform=06U%7C04%7C&mealDivList=3&searchMealDivList=3&searchKeyword=&reservationStatus=1'
# xpath_data1 = '/html/body/div[2]/div[1]/div[3]/div[2]/div[2]/div/div/ul[1]/li[1]/p'
# xpath_data1 = '/html/body/div[2]/div[1]/div[3]/div[2]/div[2]/div/div'
xpath_data1 = '/html/body/div[2]/div[1]/div[3]/div[2]/div[2]/div/div/ul[1]/li'
# xpath_data1 = '/html/body/div[2]/div[1]/div[3]/div[2]/div[2]/div/div/ul[1]'

In [158]:
browser = webdriver.Chrome(executable_path = 'D:\\Python\\lib\\chromedriver\\100\\chromedriver.exe')


In [159]:
time.sleep(3)
browser.get(url)
time.sleep(3)

In [161]:
html_s = BeautifulSoup(browser.page_source, 'html.parser')

In [162]:
lxml_coverted_data = html.fromstring(
    str(
        BeautifulSoup(browser.page_source, 'html.parser')
    )
)

data1 = lxml_coverted_data.xpath(xpath_data1)

In [164]:
data1

[<Element li at 0x15c9fe89c70>,
 <Element li at 0x15c9fe89720>,
 <Element li at 0x15c9fe89810>,
 <Element li at 0x15c9fe89860>,
 <Element li at 0x15c9fe89b80>,
 <Element li at 0x15c9fe89bd0>,
 <Element li at 0x15c9fe89950>,
 <Element li at 0x15c9fe89b30>,
 <Element li at 0x15c9fe898b0>,
 <Element li at 0x15c9fe897c0>]

In [166]:
reserve_list = [ [ y.text.replace('\n','').replace(' ','') for y in x.getchildren()] for x in data1 ]
reserve_list

[['16:00', '満席'],
 ['16:10', '満席'],
 ['16:20', '満席'],
 ['16:30', '満席'],
 ['16:40', '満席'],
 ['16:50', '満席'],
 ['17:00', '満席'],
 ['17:10', '満席'],
 ['17:20', '満席'],
 ['17:30', '満席']]

In [167]:
reserve_list_scc

[['16:00', '満席'],
 ['16:10', '満席'],
 ['16:20', '満席'],
 ['16:30', '満席'],
 ['16:40', ''],
 ['16:50', ''],
 ['17:00', '満席'],
 ['17:10', ''],
 ['17:20', ''],
 ['17:30', ''],
 ['17:40', ''],
 ['17:50', '満席'],
 ['18:00', '満席'],
 ['18:10', '満席'],
 ['18:20', '満席']]

In [168]:
browser.quit()

In [177]:
check = list([])
for ti, stt in reserve_list:
    if stt != '満席':
        check.append(ti)

if len(check) > 0:
    ok_times = ','.join(check)
    msg = f""" (test) 空きが出ました！ \n {ok_times} """
    payload = {'message': msg}
    r = requests.post(notify_url, headers=headers, params=payload,)
